In [36]:
from blpapi import *
import json
import pandas as pd
import ipywidgets
from pprint import pprint,pformat
import logging as log
from datetime import datetime,timedelta, date, time

In [37]:
log.basicConfig(
    level=log.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        log.FileHandler('bloomberg_api.log',mode='w'),  # Log to file
    ]
)
logger = log.getLogger(__name__)

In [38]:
df_source = []

In [ ]:
config = json.load(open('bpipe_config.local.json'))
pprint(config)

APP_NAME = config["appname"]

In [40]:
sessOpts = SessionOptions()

for i,v in enumerate(config["hosts"]):
    sessOpts.setServerAddress(v["addr"],v["port"],i)

In [ ]:
authOpts = AuthOptions.createWithApp(APP_NAME)
sessOpts.setSessionIdentityOptions(authOpts,correlationId=CorrelationId(APP_NAME))

In [42]:
if "tlsInfo" in config:
    tlsInfo = config["tlsInfo"]
    pk12Blob = None
    pk7Blob = None
    with open(tlsInfo['pk12path'], 'rb') as pk12File:
        pk12Blob = pk12File.read()
    with open(tlsInfo['pk7path'], 'rb') as pk7File:
        pk7Blob = pk7File.read()

    sessOpts.setTlsOptions(TlsOptions.createFromBlobs(pk12Blob, tlsInfo['password'], pk7Blob))

In [43]:
def responseHandler(msg):
    _data = {}
    for _d in msg.toPy()["DATA"]:
        for k, v in _d.items():
            # Handle datetime.date objects
            if isinstance(v, date):
                _data[k] = v.strftime('%Y/%m/%d')
            elif isinstance(v, time):
                _data[k] = v.strftime('%H:%M:%S.%f')[:-3]
            else:
                _data[k] = v

    logger.info(pformat(_data))
    return _data

In [44]:
def onEvent(event,session):
    global df_source

    eventType = event.eventType()
    
    for msg in event:
        msgType = msg.messageType()
        if msg.correlationId():
            corrVal = msg.correlationId().value()

        if eventType == Event.SUBSCRIPTION_DATA:
            logger.info(pformat(msg.toPy()))
        elif eventType == Event.RESPONSE or eventType == Event.PARTIAL_RESPONSE:
            df_source.append(responseHandler(msg))
            
        else:
            logger.info(pformat(msg.toPy()))

In [45]:
session = Session(sessOpts,onEvent)

In [ ]:
session.start()

In [ ]:
session.openService("//blp/mktlist")
session.openService("//blp/fo-discovery")

In [48]:
sublist = SubscriptionList()
secs = "IBM US Equity".split(",")

for s in secs:
    logger.info(f"subscribing to /chain/bpkbl/{s}")
    sublist.add(f"//blp/mktlist/chain/bpkbl/{s}",correlationId=CorrelationId(s))

session.subscribe(sublist)

In [49]:
for i in range(0,sublist.size()):
    session.cancel(sublist.correlationIdAt(i))

del sublist

In [ ]:
# Add this cell before the request cell
security_input = ipywidgets.Text(
    value='NKY Index',
    description='Security:',
    style={'description_width': 'initial'}
)

days_slider = ipywidgets.IntSlider(
    value=180,
    min=1,
    max=365,
    step=1,
    description='Days Forward:',
    style={'description_width': 'initial'}
)

put_call_dropdown = ipywidgets.Dropdown(
    options=['C', 'P', 'F', 'T', 'M'],
    value='C',
    description='Option Type:',
    style={'description_width': 'initial'}
)

execute_button = ipywidgets.Button(
    description='Execute Request',
    style={'description_width': 'initial'},
    button_style='primary'
)

# Create a vertical box to stack the widgets
controls = ipywidgets.VBox([
    security_input,
    days_slider,
    put_call_dropdown,
    execute_button
])
display(controls)

In [51]:
# Define the execution function
def on_execute_button_clicked(b):
    global df_source
    
    # Clear the previous data
    df_source = []
    
    svc = session.getService("//blp/fo-discovery")
    req = svc.createRequest("OptionsScreenRequest")
    
    underlying = req.getElement("SEARCH_CRITERIA").getElement("UNDERLYING")
    underlying.setElement("UNDERLYING_SECURITY", security_input.value)
    underlying.setElement("UNDERLYING_TYPE", "PARSEKYABLE_DES_SOURCE")
    
    current_date = datetime.now().strftime('%Y-%m-%d')
    end_date = (datetime.now() + timedelta(days=days_slider.value)).strftime('%Y-%m-%d')
    
    fieldFields = req.getElement("FILTER_FIELDS")
    fieldFields.setElement("OPT_EXPIRE_DT_GTEQ", current_date)
    fieldFields.setElement("OPT_EXPIRE_DT_LTEQ", end_date)
    fieldFields.setElement("OPT_PUT_CALL", put_call_dropdown.value)
    
    logger.info(req)
    df_source = []
    evtQueue = EventQueue()
    session.sendRequest(req,eventQueue=evtQueue)

    search = True
    while search:
        evt = evtQueue.nextEvent(30000)
        evtType = evt.eventType()
        for msg in evt:
            if evtType == Event.TIMEOUT:
                logger.info("Timeout occurred")
                search = False
            elif evtType == Event.RESPONSE or evtType == Event.PARTIAL_RESPONSE:
                df_source.append(responseHandler(msg))
                if evtType == Event.RESPONSE:
                    search = False
            else:
                logger.info(pformat(evt.toPy()))

    # Add a small delay to allow the data to be collected
    import time
    time.sleep(1)
    
    # Clear previous output and display new DataFrame
    from IPython.display import clear_output, display
    clear_output(wait=True)
    
    # Redisplay the controls
    display(controls)
    # Create and display the DataFrame
    if len(df_source) > 0:
        df = pd.DataFrame(df_source).sort_values(["OPT_EXPIRE_DT"], ascending=False)
        display(df[["PARSEKYABLE_DES_SOURCE","FEED_SOURCE","OPT_STRIKE_PX","OPT_EXPIRE_DT","OPT_PUT_CALL","OPT_UNDL_TICKER","FEED_EID1","FEED_EID2","FEED_EID3","FEED_EID4"]])

execute_button.on_click(on_execute_button_clicked)

In [ ]:
session.stop()